<a href="https://colab.research.google.com/github/mishafoniakov/SberTalentCase_C-2/blob/main/SberTalentCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sber Talent Case Contest 2023 C-2**

---





# 0. Подготовка рабочей среды

## 0.1. Скачивание нужных модулей

In [1]:
!pip install pandas
!pip install numpy
!pip install torch
!pip install nltk
!pip install gensim
!pip install word2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=141242 sha256=3dce0d92f56b119b32ba6fea9093ea3dc9ffa1248b077afa98aaca51a3723e25
  Stored in directory: /root/.cache/pip/wheels/6a/fa/d1/e03e8c10e0e2aa5c7b6e2b46b4a1c715d140283853937bb4b1
Successfully built word2vec


## 0.2. Запуск датасета

In [41]:
sample = pd.read_json('sample.json', encoding='utf-8')
sample.head(5)

,id,text
0,1,Ты нашёл их или нет?
1,2,Почему она так со мной поступает?
2,3,Никто туда больше не ходит.
3,4,У него с собой не было тогда денег.
4,5,Почему они с нами так поступают?


# 1. Предобрадотка данных

In [42]:
import pandas as pd
import numpy as np
import re
import torch
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import gensim.models
from pymystem3 import Mystem

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1.1. Очистка первичных данных

Необходимо проверить предложения на их типы: повествовательное, восклицательное и вопросительное. Это определяет последний знак препинания в предложении. Если предложение оканчивается на ".", то оно повествовательное, если на "!" - восклицательное, если на "?", то вопросительное.

In [43]:
sample['type'] = sample['text'].apply(lambda x: x[-1])

Затем необходимо будет очистить предложения и привести их к общему виду:

> Удалить лишние пробелы

> Убрать прописные буквы: сделать все буквы строчные

> Убрать все символы кроме цифр и букв

> Провести лемматизацию слов в предложении: привести все слова к начальной форме

> Также дополнительно провести очистку, оставив только цифры и буквы: возможно такие остались после лемматизации

> Также повторно удалить возможные возникшие дополнительные пробелы

> Очистить предложения от стоп-слов

> Провести токенизацию получившихся предложений

In [44]:
nltk.download('stopwords')
STOPWORDS = list(set(stopwords.words('russian')))
m = Mystem()

def text_prepare(txt):
    txt = re.sub('\s+', ' ', txt)
    txt = txt.replace('\\', '').lower().strip()
    txt = re.sub('[^0-9а-яА-ЯЁё ]+', '', txt)
    txt = ''.join(m.lemmatize(txt))
    txt = re.sub('[^0-9а-яА-ЯЁё ]+', '', txt)
    txt = re.sub('\s+', ' ', txt)
    return txt

def filter_stopwords(tokens):
    return [w for w in tokens if not w.lower() in STOPWORDS]

def tokenize(text):
    tokens = text.split()
    tokens = filter_stopwords(tokens)
    return tokens

sample['cleaned_text'] = sample['text'].apply(text_prepare)
sample['cleaned_text'] = sample['cleaned_text'].apply(tokenize)
sample.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,text,type,cleaned_text
0,1,Ты нашёл их или нет?,?,[находить]
1,2,Почему она так со мной поступает?,?,"[почему, поступать]"
2,3,Никто туда больше не ходит.,.,"[никто, туда, ходить]"
3,4,У него с собой не было тогда денег.,.,[деньги]
4,5,Почему они с нами так поступают?,?,"[почему, поступать]"


## 1.2. Формирование датасета для анализа

Далее необходимо получить cartesian product датасета: сделать сочетание предложений каждое с каждым и переименовать получившиеся колонки для более удобного чтения

> Делаем cross join нашего датасета

> Убираем повторяющиеся парные значения

> Выбираем интересующие нас столбцы

> Необходимо убрать пары предложений с разными типами, поскольку они не могут быть рерайтами

> Переименовываем столбцы для более удобного чтения

In [45]:
sample_cross = sample.merge(sample, how = 'cross')
sample_cross = sample_cross.loc[(sample_cross['id_x'] != sample_cross['id_y'])]
sample_cross = sample_cross[['text_x', 'text_y', 'type_x', 'type_y', 'cleaned_text_x', 'cleaned_text_y']]
sample_cross = sample_cross.rename(columns={'text_x': 'init_text', 'text_y': 'cand_text', 'type_x': 'init_type', 'type_y': 'cand_type', 'cleaned_text_x': 'init_tokens', 'cleaned_text_y': 'cand_tokens'})
sample_cross = sample_cross.loc[sample_cross['init_type'] == sample_cross['cand_type']]
sample_cross.head(5)

,init_text,cand_text,init_type,cand_type,init_tokens,cand_tokens
1,Ты нашёл их или нет?,Почему она так со мной поступает?,?,?,[находить],"[почему, поступать]"
4,Ты нашёл их или нет?,Почему они с нами так поступают?,?,?,[находить],"[почему, поступать]"
8,Ты нашёл их или нет?,Что сделал Том с деньгами?,?,?,[находить],"[сделать, деньги]"
9,Ты нашёл их или нет?,Том меня сейчас хочет видеть?,?,?,[находить],"[хотеть, видеть]"
11,Ты нашёл их или нет?,Тебе это всё нравится?,?,?,[находить],"[это, нравиться]"


# 2. Создание модели Word2Vec и алгоритма

In [46]:
from gensim.models.word2vec import Word2Vec

##2.1. Создание модели Word2Vec

Создаём модель Word2Vec c минимальным значением токенизированного предложения 1

In [47]:
model = gensim.models.Word2Vec(sample['cleaned_text'], min_count=1)

## 2.2. Создание алгоритма для поиска рерайтов

Алгоритм предлагаем следующий:

> Из полученного датасета выводим пару токенизированных предложений

> Внутри этой пары сравниваем попарно слова из одного и другого токенизированного предложения с присваиванием каждому слову вектор

> Считаем cosine similarity для такой пары

> Выводим средний cosine similarity из каждой пары токенизированных предложений

> Если средний cosine similarity больше или равен 0.3 - то это рерайт

In [48]:
n = sample_cross.shape[0]
init = sample_cross['init_tokens'].values
cand = sample_cross['cand_tokens'].values
coeff = [0] * n
for i in range(n):
    m, l = len(init[i]), len(cand[i])
    cossim_counter = 0
    for j in range(m):
        for k in range(l):
            emb_1 = torch.FloatTensor(model.wv[init[i][j]]).unsqueeze(0)
            emb_2 = torch.FloatTensor(model.wv[cand[i][k]]).unsqueeze(0)
            cossim = abs((torch.nn.functional.cosine_similarity(emb_1, emb_2)).item())
            cossim_counter += cossim
    if m * l == 0:
        coeff[i] = 0
    else:
        coeff[i] = cossim_counter / (m * l)
sample_cross['coeff'] = coeff

## 2.3. Результат

Выводим получившийся результат и сохраняем его в файл JSON

In [49]:
result = sample_cross.loc[sample_cross['coeff'] >= 0.3]
result = result[['init_text', 'cand_text', 'coeff']]
print(f'Выявлено {result.shape[0]} пар строк рерайта')
result.head(20)

Выявлено 960 пар строк рерайта


,init_text,cand_text,coeff
416,Почему она так со мной поступает?,Почему они с нами так поступают?,0.518013
426,Почему она так со мной поступает?,Почему она так с ней поступает?,0.518013
481,Почему она так со мной поступает?,Почему они так с ним поступают?,0.518013
561,Почему она так со мной поступает?,Почему он так со мной поступает?,0.518013
583,Почему она так со мной поступает?,Почему она с ним так поступает?,0.518013
645,Почему она так со мной поступает?,Почему она так с ним поступает?,0.518013
653,Почему она так со мной поступает?,Как бы ты посоветовал мне поступить?,0.338154
1089,Никто туда больше не ходит.,Никто больше туда не ходит.,0.364886
1248,У него с собой не было тогда денег.,У него тогда не было с собой денег.,1.000000
1286,У него с собой не было тогда денег.,Мне никогда не нужно было столько денег.,0.410609


In [50]:
result.to_json('word2vec.json')

# 3. Создание модели KeyedVectors и алгоритма

In [51]:
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile

## 3.1. Формирование модели Word2Vec с помощью модуля **KeyedVectors**

Создаём модель KeyedVectors для построения векторов

In [52]:
kmodel = gensim.models.Word2Vec(sample['cleaned_text'], min_count=1)
word_vectors = kmodel.wv
fname = get_tmpfile('vectors.kv')
word_vectors.save(fname)
word_vectors = KeyedVectors.load(fname, mmap='r')

## 3.2. Создание алгоритма для поиска рерайтов

Алгоритм предлагается следующий:

> Из полученного датасета из каждого предложения каждой пары берём поэлементное среднее и объединяем средние значения

> Получаем числовой вектор из каждого предложения

> Считаем cosine similarity для каждой пары предложений

> Если средний cosine similarity больше или равен 0.3 - то это рерайт

In [53]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = word2vec.vectors.shape[1]

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec.key_to_index]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

vectorizer = MeanEmbeddingVectorizer(word_vectors)

n = sample_cross.shape[0]
init = sample_cross['init_tokens'].values
cand = sample_cross['cand_tokens'].values
coeff = [0] * n
for i in range(n):
    emb_init = torch.FloatTensor(vectorizer.transform([init[i]]))
    emb_cand = torch.FloatTensor(vectorizer.transform([cand[i]]))
    coeff[i] = abs(torch.nn.functional.cosine_similarity(emb_init, emb_cand).item())
sample_cross['coeff'] = coeff

##3.3. Результат

Выводим получившийся результат и сохраняем его в файл JSON

In [54]:
result = sample_cross.loc[(sample_cross['init_type'] == sample_cross['cand_type']) & (sample_cross['coeff'] >= 0.3)]
result = result[['init_text', 'cand_text', 'coeff']]
print(f'Выявлено {result.shape[0]} пар строк рерайта')
result.head(20)

Выявлено 5396 пар строк рерайта


,init_text,cand_text,coeff
416,Почему она так со мной поступает?,Почему они с нами так поступают?,1.000000
426,Почему она так со мной поступает?,Почему она так с ней поступает?,1.000000
476,Почему она так со мной поступает?,Почему все на нас смотрят?,0.542886
481,Почему она так со мной поступает?,Почему они так с ним поступают?,1.000000
561,Почему она так со мной поступает?,Почему он так со мной поступает?,1.000000
583,Почему она так со мной поступает?,Почему она с ним так поступает?,1.000000
645,Почему она так со мной поступает?,Почему она так с ним поступает?,1.000000
653,Почему она так со мной поступает?,Как бы ты посоветовал мне поступить?,0.371515
721,Почему она так со мной поступает?,Почему никто нам не помогает?,0.352807
791,Почему она так со мной поступает?,Почему всн на нас смотрят?,0.456899


In [55]:
result.to_json('keyedvectors.json')